In [1]:
import pandas as pd

## Load data

#### Mobile Log data

In [2]:
# Mobile
mobile = pd.read_csv("./data(다운로드금지)/PCMB공통수집패널로그_MB_201406.csv", low_memory=False, header=None)

In [3]:
mobile.columns = ["UID","Full_URL","Site","Domain","Time", "Title",
              "keyword_p","keyword_t",
              "category_1","category_2","category_3","category_4","category_5","category_6",
              "ownership_1","ownership_2"]

In [4]:
mobile.UID = mobile.UID.map(lambda x: x.replace(" ",""))

#### PC Log data

In [5]:
# PC
pc1 = pd.read_csv("/home/da/jupyter/김현준/PCMB공통수집패널로그_PC_20140601_10.csv", header=None, low_memory=False)
pc2 = pd.read_csv("/home/da/jupyter/김현준/PCMB공통수집패널로그_PC_20140611_20.csv", header=None, low_memory=False)
pc3 = pd.read_csv("/home/da/jupyter/김현준/PCMB공통수집패널로그_PC_20140621_30.csv", header=None, low_memory=False)

In [6]:
pc1.columns = ["UID","Full_URL","Site","Domain","Time", "Title",
              "keyword_p","keyword_t",
              "category_1","category_2","category_3","category_4","category_5","category_6",
              "ownership_1","ownership_2"]
pc2.columns = ["UID","Full_URL","Site","Domain","Time", "Title",
              "keyword_p","keyword_t",
              "category_1","category_2","category_3","category_4","category_5","category_6",
              "ownership_1","ownership_2"]
pc3.columns = ["UID","Full_URL","Site","Domain","Time", "Title",
              "keyword_p","keyword_t",
              "category_1","category_2","category_3","category_4","category_5","category_6",
              "ownership_1","ownership_2"]

In [8]:
pc1.UID = pc1.UID.map(lambda x: x.replace(" ",""))
pc2.UID = pc2.UID.map(lambda x: x.replace(" ",""))
pc3.UID = pc3.UID.map(lambda x: x.replace(" ",""))

#### Survey data

In [9]:
# 패널 서베이
survey = pd.read_excel("./data(다운로드금지)/140716_SSK 구매행태 조사 Raw Data_F.xlsx")
survey.UID = survey.UID.map(lambda x: x.replace(" ",""))
print("서베이의 총 UID 개수:", survey.UID.nunique())

서베이의 총 UID 개수: 1066


## 서베이 응답 기준으로 UID 추출

* 가전제품 구매
* 6월 기준, 구매 이력이 있음
* 인터넷에서 제품 정보를 얻은 경우

In [10]:
가전구매 = survey[(survey["A3_1"]==1)].copy()
print(len(가전구매))

538


In [11]:
가전구매_6월 = 가전구매[가전구매["A4_2_1"]==3].copy()
print(len(가전구매_6월))

186


In [12]:
가전구매_6월_제품정보 = 가전구매_6월[가전구매_6월["A4_5"].isin([3,4,5,6])].copy()
print(len(가전구매_6월_제품정보))

120


In [13]:
sample_uid = 가전구매_6월_제품정보["UID"].unique().tolist()

## Mobile/PC 로그에서 해당 UID 로그만 추출

In [14]:
mob_sample = mobile[mobile["UID"].isin(sample_uid)].copy()

In [15]:
pc1_sample = pc1[pc1["UID"].isin(sample_uid)].copy()
pc2_sample = pc2[pc2["UID"].isin(sample_uid)].copy()
pc3_sample = pc3[pc3["UID"].isin(sample_uid)].copy()

pc_sample = pd.concat([pc1_sample, pc2_sample, pc3_sample], axis=0)

In [16]:
# Mobile/PC 여부 컬럼 추가
# 1 = PC
# 0 = Mobile
mob_sample["PC"] = 0
pc_sample["PC"] = 1

## Mobile/PC to one dataframe

In [17]:
sample = pd.concat([pc_sample, mob_sample], axis=0)

In [23]:
# 데이터프레임 인덱스 재정리
sample.reset_index(drop=True, inplace=True)

In [18]:
# Time 데이터 속성 변경
sample["Time"] = sample["Time"].astype("datetime64")

In [19]:
# 안쓰는 컬럼 삭제
sample.drop(["Site","Domain","Title","keyword_p","keyword_t",
             "category_1", "category_2", "category_3", "category_4", "category_5", "category_6",
             "ownership_1", "ownership_2"], axis=1, inplace=True)

In [20]:
print("# of rows:", len(sample))

# of rows: 3519992


In [21]:
print("# of unique UIDs:", sample.UID.nunique())

# of unique UIDs: 120


In [24]:
sample.head()

,UID,Full_URL,Time,PC
0,20130603169E3A2A5EA7,http://pluszone.gmarket.co.kr/,2014-06-01 00:33:26,1
1,20130603169E3A2A5EA7,http://pluszone.gmarket.co.kr/eventzone/PopChe...,2014-06-01 00:33:28,1
2,20130603169E3A2A5EA7,http://pluszone.gmarket.co.kr/favicon.ico,2014-06-01 00:33:28,1
3,20130603169E3A2A5EA7,http://promotion.gmarket.co.kr/eventzone/GStam...,2014-06-01 00:33:16,1
4,20130603169E3A2A5EA7,http://promotion.gmarket.co.kr/eventzone/GStam...,2014-06-01 00:33:22,1


In [ ]:
# 데이터프레임 저장
sample.to_csv("./data(다운로드금지)/0109_PC&Mob_06_가전제품구매.csv", index=False)